<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# plusieurs sortes de merge

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## reprenons ..

si vous avez bien suivi et exécuté ce qui précède, vous devez avoir un répertoire `my-first-repo`:

* qui contient 4 commits
* et deux branches `master` et `devel`
* la branche `master` pointe sur notre premier merge
* la branche `devel` est un peu en retrait

In [ ]:
# si nécessaire, vous pouvez remettre le repository en l'état
# 
# pour cela mettez "true" au lieu de ""
# et bien sûr évaluer la cellule

reset=""

if [ -n "$reset" ]; then 
    cd $TOP
    bash $SCRIPTS/2-01-my-first-repo.sh
    bash $SCRIPTS/2-02-my-first-changes.sh
    bash $SCRIPTS/2-03-my-first-branch.sh
fi >& /dev/null

## le point..

In [ ]:
# si nécessaire, on se place dans le repo git
[ -d my-first-repo ] && cd my-first-repo

pwd

In [ ]:
# vous devez avoir deux branches, 6 commits dont un merge
# et être sur la branche master 
git l --all

## commits comparables ou pas

* la relation *commit X référence commit Y* définit un **ordre partiel** sur les commits

* selon que deux commits **sont comparables ou non**
  * la création d'un merge va avoir des effets très différents

`master` et `media` non comparables `master` $\nless$ `devel`  
`master` $\ngtr$ `devel`  



![](media/order-2-not-compare.png) 

ils sont comparables  
`master` > `media`

![](media/order-3-compare.png) 

## deux sortes de merge

* si les deux points du merge **sont comparables** :
  * **pas besoin** de créer un commit !
* dans le cas contraire
  * un nouveau commit **est nécessaire**
  
* voyons ces deux cas plus en détail  

## (1) merge avec deux commits comparables

si les deux commits sont comparables

* le plus grand des deux **contient déjà** les changements communs aux deux commits
* d'ailleurs: le plus proche ancêtre commun est le plus petit des deux


## (1) merge avec deux commits comparables

un merge dit *fast-forward*

![](media/merge-1-fast-forward.png)

un merge sans aucun effet

![](media/merge-2-noop.png)

## (2) merge avec commits incomparables

* dans ce cas donc, il faut **créer un commit** qui incorpore les changements
* c'est fait **automatiquement** par `git merge` 
* toutefois :
  * l'algorithme fonctionne à base de `diff`
  * qui est **orienté lignes**
  * d'où la bien meilleure adéquation sur **le texte**

digression:

* c'est une des raisons qui ont favorisé le format *markdown* 
* on en reparlera

## notion de conflit

* imaginez que vous avez une section de code, disons une ligne
* qui est changée dans la branche A
* et **aussi** changée dans la branche B
* mais de manière différente..

## en cas de conflit

Dans ce genre de cas

* la fusion automatique **ne va pas réussir**
* et le commit **ne sera pas créé**

`git merge` va

* fusionner le maximum
  * c'est mis dans l'index
* pour le reste
  * il insère des balises dans le texte
  * avec les deux versions
  * **pas dans l'index**
  * vous laisse le soin de choisir
* il vous reste à 
  * mettre dans l'index les conflits résolus
  * créer le commit vous même

## auto merge

![](media/auto-merge-failed.png)

## exemple 1: fast-forward

In [ ]:
git l

* rappel: `master` a fusionné `devel`
* vérifiez que `master` et `devel` sont comparables
* que se passe-t-il si on merge `master` dans `devel` ?

In [ ]:
git checkout devel
git merge master

In [ ]:
git l

Réponse: 

* pas de commit créé
* `devel` "rattrape" simplement `master`

## exemple 2 : merge avec conflit - première branche

In [ ]:
# un changement qui
# ne sera pas conflictuel

$SCRIPTS/do no-worries-1

In [ ]:
# celui-ci par contre le sera

$SCRIPTS/do conflict-1

In [ ]:
git diff

## exemple 2 : merge avec conflit - première branche

In [ ]:
git add factorial.md
git commit -m 'pour conflit dans devel'

In [ ]:
git l --all

## exemple 2 : merge avec conflit - deuxième branche

In [ ]:
# remettons-nous au commit précédent
git checkout master

In [ ]:
# même logique, on fait deux changements

$SCRIPTS/do no-worries-2
$SCRIPTS/do conflict-2

In [ ]:
git diff

In [ ]:
git add factorial.py factorial.md

git commit -m'pour conflit, dans master'

## exemple 2 : merge avec conflit - on a deux branches

In [ ]:
git l --all


## exemple 2 : le merge échoue

In [ ]:
# on est sur master
git merge devel

## exemple 2 : le résultat du merge qui a échoué

In [ ]:
git status

In [ ]:
cat factorial.md

## exemple 2 : résoudre le conflit

* modifier à la main le fichier concerné
* ajouter ce changement à l'index
* on peut alors commiter

In [ ]:
# je simule une modification sous éditeur
$SCRIPTS/do resolve-conflict

cat factorial.md

In [ ]:
# pas de changement naturellement
git status

## exemple 2 : résoudre le conflit (2)

In [ ]:
# maintenant on peut mettre 
# la résolution du conflit dans l'index
git add factorial.md

In [ ]:
# plus de souci
git status

In [ ]:
# et à présent on peut committer
git commit -m  'conflit résolu'

In [ ]:
git l --all

## exercice 1

je vous invite à vous amuser à faire les diffs dans tous les sens :

* calculer le sha-1 du dernier point de fourche ("*mon premier merge*")
* diff avec HEAD (a.k.a. master)
* diff avec devel
* diff entre master et devel

In [ ]:
git diff devel master


## exercice 2

s'entraîner à faire des merge 

1. avec des configurations *claires*, 
   c'est-à-dire où clairement les deux changements sont indépendants les uns des autres
   
1. on a vu un conflit quand les deux branches **modifient** la même ligne  
   que se passe-t-il si les deux branches **insèrent** du code au même endroit dans un fichier

## résumé

* lorsque deux commits sont comparables
  * c'est-à-dire qu'il existe un chemin uniquement descendant de l'un à l'autre
  * alors un merge **ne produit pas de commit**
  * mais peut provoquer un "rattrapage" d'une branche par une autre

* dans le cas contraire
  * un merge **crée un commit** de fusion
  * sauf lorsqu'il y a conflit
  * dans ce cas c'est à vous de résoudre ces conflits à la main
  * et de terminer le travail en créant le commit  

## état

comme d'habitude nous observons notre répo à ce stade

In [ ]:
git l --all